# Keras on Session mode

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

import numpy as np
import matplotlib.pyplot as plt
import os

#import tensorflow.contrib.eager as tfe
#tf.enable_eager_execution()

In [2]:
learning_rate = 0.001
training_epochs = 21
batch_size = 100

In [3]:
cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'mnist_keras_session'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir_name)

## TensorBoard
log_dir = os.path.join(cur_dir, 'tensorboard', model_dir_name)
os.makedirs(log_dir, exist_ok=True)

In [4]:
mnist = keras.datasets.mnist

(train_x, train_y), (test_x, test_y) = mnist.load_data()
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [5]:
train_x = train_x.astype(np.float32) / 255
test_x = test_x.astype(np.float32) / 255

train_x = np.reshape(train_x, [-1,28*28])
test_x = np.reshape(test_x, [-1,28*28])

train_y = to_categorical(train_y, 10) ## one-hot encoding
test_y = to_categorical(test_y,10)

# tf.data.Dataset

train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(buffer_size=70000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(batch_size)

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(60000, 784) (60000, 10) (10000, 784) (10000, 10)


## Sequential

In [ ]:
def create_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(256, activation=tf.nn.relu, input_shape=(784,)))  ## input_shape at first layer appended to Sequantial
    model.add(keras.layers.Dense(10))
    return model

In [ ]:
model = create_model()

In [ ]:
model.summary()

In [6]:
def loss_fn(model, x, labels):
    logits = model(x)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    return loss

def evaluate(model, x, labels):
    logits = model(x)
    correct_pred = tf.equal(tf.argmax(logits,1), tf.argmax(labels,1))
    acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    return acc

X = tf.placeholder(tf.float32, shape=[None, 784])
Y = tf.placeholder(tf.float32, shape=[None, 10])

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=0.01)

checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)

In [ ]:
loss = loss_fn(model, X, Y)

simple_optm = optimizer.minimize(loss)

tr_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
grads_and_vars = optimizer.compute_gradients(loss, var_list = tr_vars)
optm = optimizer.apply_gradients(grads_and_vars)

accuracy = evaluate(model, X, Y)

In [21]:
init = tf.global_variables_initializer()
num_train_data = train_x.shape[0]
num_test_data = test_x.shape[0]

train_iterator = train_dataset.make_initializable_iterator()
test_iterator = test_dataset.make_initializable_iterator()

tr_x, tr_y = train_iterator.get_next()
ts_x, ts_y = test_iterator.get_next()

In [ ]:
with tf.Session() as sess:
    
    status=checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
    status.initialize_or_restore(sess)


    for epoch in range(training_epochs):
        avg_loss = 0
        avg_train_acc = 0
        avg_test_acc = 0
        train_step = 0
        test_step = 0

        sess.run(train_iterator.initializer)
        for _ in range(num_train_data//batch_size):
            train_input, train_label = sess.run([tr_x, tr_y])
            step_loss, step_accuracy, _ = sess.run([loss, accuracy, optm], feed_dict={X:train_input, Y: train_label})
            avg_loss += step_loss
            avg_train_acc += step_accuracy
            train_step += 1
        avg_loss /= train_step
        avg_train_acc /= train_step
        
        print(f"#EPOCH: {epoch+1}")
        print(f"train_avg_loss:{avg_loss}\ttrain_avg_accuracy:{avg_train_acc}")
        
        if epoch%5 == 0:
            sess.run(test_iterator.initializer)
            for _ in range(num_test_data//batch_size):
                test_x, test_y = sess.run([ts_x, ts_y])
                step_accuracy = sess.run(accuracy, feed_dict={X:test_x, Y:test_y})
                avg_test_acc += step_accuracy
                test_step += 1 
            avg_test_acc /= test_step
        
            print(f"test_avg_accuracy:{avg_test_acc}")
            
            checkpoint.save(file_prefix=checkpoint_prefix)
    

## functional model

In [ ]:
def create_model_func():
    with tf.variable_scope('functional', reuse=False):
        inputs =  keras.Input(shape=(784,))
        fc_layer1 = keras.layers.Dense(256, activation = tf.nn.relu)(inputs)
        logits = keras.layers.Dense(10)(fc_layer1)
        return keras.Model(inputs=inputs, outputs=logits)

model_func = create_model_func()

In [ ]:
model_func.summary()

In [ ]:
cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'mnist_keras_functional_session'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir_name)

## TensorBoard
log_dir = os.path.join(cur_dir, 'tensorboard', model_dir_name)
os.makedirs(log_dir, exist_ok=True)

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=0.01)
checkpoint = tf.train.Checkpoint(model_func=model_func, optimizer=optimizer)

loss_func = loss_fn(model_func, X, Y)

simple_optm = optimizer.minimize(loss_func)


tr_vars_func = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='functional')
grads_and_vars_func = optimizer.compute_gradients(loss_func, var_list = tr_vars_func)
optm_func = optimizer.apply_gradients(grads_and_vars_func)

accuracy_func = evaluate(model_func, X, Y)

In [ ]:
with tf.Session() as sess:
    
    status = checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
    status.initialize_or_restore(sess)

    for epoch in range(training_epochs):
        avg_loss = 0
        avg_train_acc = 0
        avg_test_acc = 0
        train_step = 0
        test_step = 0

        sess.run(train_iterator.initializer)
        for _ in range(num_train_data//batch_size):
            train_input, train_label = sess.run([tr_x, tr_y])
            step_loss, step_accuracy, _ = sess.run([loss_func, accuracy_func, optm_func], feed_dict={X:train_input, Y: train_label})
            avg_loss += step_loss
            avg_train_acc += step_accuracy
            train_step += 1
        avg_loss /= train_step
        avg_train_acc /= train_step
        
        print(f"#EPOCH: {epoch+1}")
        print(f"train_avg_loss:{avg_loss}\ttrain_avg_accuracy:{avg_train_acc}")
        
        if epoch%5 == 0:
            sess.run(test_iterator.initializer)
            for _ in range(num_test_data//batch_size):
                test_x, test_y = sess.run([ts_x, ts_y])
                step_accuracy = sess.run(accuracy_func, feed_dict={X:test_x, Y:test_y})
                avg_test_acc += step_accuracy
                test_step += 1 
            avg_test_acc /= test_step
        
            print(f"test_avg_accuracy:{avg_test_acc}")
            
            checkpoint.save(file_prefix=checkpoint_prefix)

## Class model

In [7]:
class ClassModel(keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.fc_layer1 = keras.layers.Dense(256, activation='relu')
        self.logits = keras.layers.Dense(10)
        
    def call(self, inputs):
        net = self.fc_layer1(inputs)
        net = self.logits(net)
        return net
        
with tf.variable_scope('class_based',use_resource=True):
    model_class = ClassModel()

In [8]:
cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'mnist_keras_class_session'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir_name)

## TensorBoard
log_dir = os.path.join(cur_dir, 'tensorboard', model_dir_name)
os.makedirs(log_dir, exist_ok=True)

In [17]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=0.01)
checkpoint = tf.train.Checkpoint(model_class=model_class, optimizer=optimizer)

loss_class = loss_fn(model_class, X, Y)

In [18]:
model_class.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  200960    
_________________________________________________________________
dense_1 (Dense)              multiple                  2570      
Total params: 203,530
Trainable params: 203,530
Non-trainable params: 0
_________________________________________________________________


In [24]:
simple_optm = optimizer.minimize(loss_class)


tr_vars_class = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='class_model')
grads_and_vars_class = optimizer.compute_gradients(loss_class, var_list = tr_vars_class)
optm_class = optimizer.apply_gradients(grads_and_vars_class)

accuracy_class = evaluate(model_class, X, Y)

In [25]:
tr_vars_class

[<tf.Variable 'class_model/dense/kernel:0' shape=(784, 256) dtype=float32>,
 <tf.Variable 'class_model/dense/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'class_model/dense_1/kernel:0' shape=(256, 10) dtype=float32>,
 <tf.Variable 'class_model/dense_1/bias:0' shape=(10,) dtype=float32>]

### tf.variable_scope does not work for class-based keras model!!!

In [27]:
with tf.Session() as sess:
    
    status=checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
    status.initialize_or_restore(sess)

    for epoch in range(training_epochs):
        avg_loss = 0
        avg_train_acc = 0
        avg_test_acc = 0
        train_step = 0
        test_step = 0

        sess.run(train_iterator.initializer)
        for _ in range(num_train_data//batch_size):
            train_input, train_label = sess.run([tr_x, tr_y])
            step_loss, step_accuracy, _ = sess.run([loss_class, accuracy_class, optm_class], feed_dict={X:train_input, Y: train_label})
            avg_loss += step_loss
            avg_train_acc += step_accuracy
            train_step += 1
        avg_loss /= train_step
        avg_train_acc /= train_step
        
        print(f"#EPOCH: {epoch+1}")
        print(f"train_avg_loss:{avg_loss}\ttrain_avg_accuracy:{avg_train_acc}")
        
        if epoch%5 == 0:
            sess.run(test_iterator.initializer)
            for _ in range(num_test_data//batch_size):
                test_x, test_y = sess.run([ts_x, ts_y])
                step_accuracy = sess.run(accuracy_class, feed_dict={X:test_x, Y:test_y})
                avg_test_acc += step_accuracy
                test_step += 1 
            avg_test_acc /= test_step
        
            print(f"test_avg_accuracy:{avg_test_acc}")
            
            checkpoint.save(file_prefix=checkpoint_prefix)
    

#EPOCH: 1
train_avg_loss:0.04138871586571137	train_avg_accuracy:0.989616675277551
test_avg_accuracy:0.979500008225441
#EPOCH: 2
train_avg_loss:0.0391157097571219	train_avg_accuracy:0.9905000087618828
#EPOCH: 3
train_avg_loss:0.03695347749395296	train_avg_accuracy:0.9911833412448565
#EPOCH: 4
train_avg_loss:0.03533846121281385	train_avg_accuracy:0.9914666741093
#EPOCH: 5
train_avg_loss:0.03340715296411266	train_avg_accuracy:0.9921833402911822
#EPOCH: 6
train_avg_loss:0.031729518667949985	train_avg_accuracy:0.9929333397746086
test_avg_accuracy:0.9800000065565109
#EPOCH: 7
train_avg_loss:0.029954073789995164	train_avg_accuracy:0.9936166724562645
#EPOCH: 8
train_avg_loss:0.028584290660219266	train_avg_accuracy:0.9937666725118955
#EPOCH: 9
train_avg_loss:0.02721952805101561	train_avg_accuracy:0.9943000052372615
#EPOCH: 10
train_avg_loss:0.02583166576611499	train_avg_accuracy:0.9945333384474119
#EPOCH: 11
train_avg_loss:0.024644941174968456	train_avg_accuracy:0.9951166712244351
test_avg_accu

## Custom Layer
- inherit keras.layers.Layer
- implement following methods
1. build (add_weight)
2. call
3. compute_output_shape
4. get_config (optionally)